### 1. Importing libraries / Data Loading and Splitting



In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import matplotlib.pyplot as plt

from torchvision import datasets, models, transforms
from os import rename, listdir, path
import math
import shutil

torch.manual_seed(1) # set the random seed

from google.colab import drive
drive.mount('/content/gdrive', force_remount=True) #Must have folder inside drive called PokeScanner

data_transform = transforms.Compose([
    # you can add other transformations in this list
    transforms.ToTensor()
])
train_transform = transforms.Compose([
    # you can add other transformations in this list
    transforms.ToTensor(),
    transforms.ColorJitter(0.1,0.1,0.1,0.1),
    transforms.RandomPerspective(0.25)
])

In [ ]:
use_cuda = True
# filePath = '/content/gdrive/MyDrive/PokeScanner' #Everyone has had this folder shared with them, Right click the folder and click "Add shortcut to Drive"

# i = 0
# j = 0
# items = listdir("/content/gdrive/MyDrive/PokeScanner/PokemonDataCleaned")
# print(items)
# for i in range (0, len(items)):
#   if (os.path.isdir("/content/gdrive/MyDrive/PokeScanner/PokemonDataCleaned/"+ items[i])):
#     dirItems = listdir("/content/gdrive/MyDrive/PokeScanner/PokemonDataCleaned/"+ items[i])
#     trainingNumber = math.floor(len(dirItems) * 0.80) 
#     validationNumber = math.floor(len(dirItems) * 0.10)
#     for j in range (0, len(dirItems)):
#      if (os.path.isfile("/content/gdrive/MyDrive/PokeScanner/PokemonDataCleaned/" + items[i] + "/" + dirItems[j])):
#        if (j <= trainingNumber):
#         shutil.copyfile("/content/gdrive/MyDrive/PokeScanner/PokemonDataCleaned/" + items[i] + "/" + dirItems[j], "/content/gdrive/MyDrive/PokeScanner/Training/" + items[i] + "/" + dirItems[j])
#        elif (j <= trainingNumber + validationNumber):
#         shutil.copyfile("/content/gdrive/MyDrive/PokeScanner/PokemonDataCleaned/" + items[i] + "/" + dirItems[j], "/content/gdrive/MyDrive/PokeScanner/Validation/" + items[i] + "/" + dirItems[j])
#        else:
#         shutil.copyfile("/content/gdrive/MyDrive/PokeScanner/PokemonDataCleaned/" + items[i] + "/" + dirItems[j], "/content/gdrive/MyDrive/PokeScanner/Testing/" + items[i] + "/" + dirItems[j])

# train_set = datasets.ImageFolder("/content/gdrive/MyDrive/PokeScanner/Training")
# validation_set = datasets.ImageFolder("/content/gdrive/MyDrive/PokeScanner/Validation")
# testing_set = datasets.ImageFolder("/content/gdrive/MyDrive/PokeScanner/Testing")

# print('Number of Training Images:', len(train_set))
# print('Number of Validation Images:', len(validation_set))
# print('Number of Testing Images:', len(testing_set))

In [ ]:
filePath = '/content/gdrive/MyDrive/PokeScanner' #Everyone has had this folder shared with them, Right click the folder and click "Add shortcut to Drive"
train_set = datasets.ImageFolder(filePath+'/Training', transform=train_transform)
validation_set = datasets.ImageFolder(filePath+'/Validation', transform=data_transform)
testing_set = datasets.ImageFolder(filePath+'/Testing', transform=data_transform)

###2. Generating More Data

### 3. Model Building and Sanity Checking

Build a convolutional neural network model that takes the (224x224 RGB) image as input, and predicts the pokemon

In [ ]:
class PokemonClassifier(nn.Module):
      def __init__(self, kernel_size):
          super(PokemonClassifier, self).__init__()
          self.conv1 = nn.Conv2d(3, 5, kernel_size) #in_channels, out_chanels, kernel_size
          self.dropout = nn.Dropout(0.1)#dropout layer
          self.pool = nn.MaxPool2d(2, 2) #kernel_size, stride 
          self.conv2 = nn.Conv2d(5, 10, kernel_size) #in_channels, out_chanels, kernel_size 
          self.conv3 = nn.Conv2d(7, 10, kernel_size) #in_channels, out_chanels, kernel_size
          shape = math.floor((251 - kernel_size - 2) / 2) + 1
          shape2 = math.floor((shape - kernel_size - 1) / 2) + 1
          shape3 = math.floor((shape2 - kernel_size - 1) / 2) + 1
          self.fc1 = nn.Linear(10 * shape2 * shape2, 200) #needs input-dependent value
          self.fc2 = nn.Linear(200, 150)

      def forward(self, x):
          x = self.pool(F.relu(self.conv1(x)))
          #print(x.shape)
          x = self.pool(F.relu(self.conv2(x)))
          #print(x.shape)
          x = self.pool(F.relu(self.conv3(x)))
          #print(x.shape)
          x = x.view(-1, 10 * x.shape[2] * x.shape[2])
          #print(x.shape)
          x = self.dropout(x)
          x = F.relu(self.fc1(x))
          #print(x.shape)
          x = self.dropout(x)
          x = self.fc2(x)
          #print(x.shape)
          return x

In [ ]:
#From tut3a
def get_accuracy(model, accuracy_data, batch_size):
    correct = 0
    total = 0
    for imgs, labels in torch.utils.data.DataLoader(accuracy_data, batch_size=batch_size):
        
        
        #############################################
        #To Enable GPU Usage
        if use_cuda and torch.cuda.is_available():
          imgs = imgs.cuda()
          labels = labels.cuda()
        else:
          print("get_accuracy: no cuda")
        #############################################
        
        
        output = model(imgs)
        
        #select index with maximum prediction score
        pred = output.max(1, keepdim=True)[1]
        correct += pred.eq(labels.view_as(pred)).sum().item()
        total += imgs.shape[0]
    return correct / total

In [ ]:
#From tut3a
def train(model, data, size, num_epochs, learning_rate):
    train_loader = torch.utils.data.DataLoader(data, batch_size=size, shuffle=True)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

    iters, losses, train_acc, val_acc, epochList = [], [], [], [], []

    # training
    n = 0 # the number of iterations
    for epoch in range(num_epochs):
        for imgs, labels in iter(train_loader):

            #############################################
            #To Enable GPU Usage
            if use_cuda and torch.cuda.is_available():
              imgs = imgs.cuda()
              labels = labels.cuda()
            else:
              print("Train: no cuda")
            #############################################
            # print("img: ", imgs.shape)
            # print("label: ", labels)
            out = model(imgs)             # forward pass
            
            loss = criterion(out, labels) # compute the total loss
            loss.backward()               # backward pass (compute parameter updates)
            optimizer.step()              # make the updates for each parameter
            optimizer.zero_grad()         # a clean up step for PyTorch

            # save the current training information
            iters.append(n)
            losses.append(float(loss)/size)             # compute *average* loss
            n += 1
        train_acc.append(get_accuracy(model, accuracy_data=train_set, batch_size=size)) # compute training accuracy 
        val_acc.append(get_accuracy(model, accuracy_data=validation_set, batch_size=size))  # compute validation accuracy
        epochList.append(epoch)
        print("Epoch {}: Training Accuracy: {} Validation Accuracy: {}".format(epoch, train_acc[epoch], val_acc[epoch]))
    
    # plotting
    plt.title("Training Curve")
    plt.plot(iters, losses, label="Train")
    plt.xlabel("Iterations")
    plt.ylabel("Loss")
    plt.show()

    plt.title("Training Curve")
    plt.plot(epochList, train_acc, label="Train")
    plt.plot(epochList, val_acc, label="Validation")
    plt.xlabel("Iterations")
    plt.ylabel("Training Accuracy")
    plt.legend(loc='best')
    plt.show()

    print("Final Training Accuracy: {}".format(train_acc[-1]))
    print("Final Validation Accuracy: {}".format(val_acc[-1]))
  


In [ ]:
model = PokemonClassifier(3)

#proper model
#############################################
#To Enable GPU Usage
if use_cuda and torch.cuda.is_available():
  model.cuda()
  print("cuda")
#############################################

#Basic first attempt at training
train(model, train_set, size=64, num_epochs=50, learning_rate=0.006)